# Data preparation

In [ ]:
import pandas as pd
df_python = pd.read_csv('data\python_posts.csv')
df_python['tag'] = 0
df_java = pd.read_csv('data\java_posts.csv')
df_java['tag'] = 1
frames = [df_python, df_java]
df = pd.concat(frames)
df = df.dropna()
df.head()

## create tokens

In [ ]:
from nltk.tokenize import WhitespaceTokenizer
tokenizer = WhitespaceTokenizer()

def tokenize(row):
    row['tokens'] = tokenizer.tokenize(row['content'])
    return row

df = df.apply(tokenize, axis=1)
df.head()
# tokenizer.tokenize("zzz zz")

## remove stopwords

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(row):
    stop_words = stopwords.words('english')
    row['tokens'] = [w for w in row['tokens'] if not w in stop_words] 
    return row
    
df = df.apply(remove_stopwords, axis=1)
df.head()

## lowercase and stemming

In [ ]:
import nltk

porter = nltk.PorterStemmer()

def simplify(row):
    row['tokens'] = [porter.stem(token) for token in row['tokens']] 
    return row
    
# df = df.apply(simplify, axis=1)
# df.head()

# Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
# vectorizer = HashingVectorizer(alternate_sign=False, norm='l2', binary=False)
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, use_idf=True)
df['features'] = df.apply(lambda row: ' '.join(row['tokens']), axis=1)
X = vectorizer.fit_transform(df['features'])
X.shape

In [ ]:
from sklearn.model_selection import train_test_split 
y = df['tag']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Training

In [ ]:
from sklearn.naive_bayes import GaussianNB
bayes = GaussianNB()
model = bayes.fit(X_train.toarray(), y_train)

# Scoring

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test.toarray())
print(classification_report(y_test, y_pred))

# Clustering

## Normalization

In [ ]:
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

svd = TruncatedSVD(50)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
# lsa = make_pipeline(normalizer)


# X_norm = lsa.fit_transform(X)
X_norm = X


## Clustering work

In [ ]:
NO_OF_CLUSTERS=4
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=NO_OF_CLUSTERS, init='k-means++', max_iter=100, n_init=50, random_state=0, n_jobs=-1, verbose=2).fit(X_norm)
y_pred = kmeans.predict(X_norm)
# y_pred_proba = kmeans.predict_proba(X_norm)

from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred)

## Centroids

In [ ]:

NO_OF_CLUSTERS=4
from wordcloud import WordCloud
import matplotlib.pyplot as plt
centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
# centroids
terms = vectorizer.get_feature_names()
for n in range(NO_OF_CLUSTERS):
    print("cluster %d" % n)
    elems = []
    for index in centroids[n, :20]:
        elems.append(terms[index])
    wc = WordCloud().generate(' '.join(elems))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    plt.show()
#     wc.show()
# kmeans.cluster_centers_
# X_norm

# Visualization

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

tsne = TSNE(n_components=2, init='pca', random_state=0)
X_tsne = tsne.fit_transform(X_norm.toarray())
df['x-tsne'] = X_tsne[:,0]
df['y-tsne'] = X_tsne[:,1]
df['prediction'] = y_pred
from ggplot import *
chart = ggplot( df, aes(x='x-tsne', y='y-tsne', color='prediction'))  + geom_point(size=100,alpha=0.1)  + scale_color_hue(l=40, c=35)
chart

In [ ]:
chart = ggplot( df, aes(x='x-tsne', y='y-tsne', color='prediction'))  + geom_point(size=100,alpha=0.1)  + scale_color_brewer(type='diverging', palette=4)

chart